In [1]:
import struct
import binascii
import requests
import scipy

import scipy.misc
import scipy.cluster

import numpy as np
import pandas as pd

from PIL import Image, ImageChops
from io import BytesIO
from PIL import Image

### **1. Read in the link csv**

In [2]:
pwd

'/Users/rosiebabbra/Desktop/side-projects/hallmart-card-color-analysis/viz'

In [2]:
df = pd.read_csv('../csvs/hallmark_image_s3_urls.csv')

### **2. Get the most common color in each card**

In [3]:
def get_pil_img(url):
    
    """ Returns the image from the URL in bytes form """
    
    response = requests.get(url)
    im = Image.open(BytesIO(response.content))
    
    return im

### **3. Get rid of whitespace in all cards**

In [5]:
def get_rid_of_whitespace(im):
    
    """ Gets rid of any white in an image, returns the image """
    
#     im = get_pil_img(url)

    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    
    if bbox:
        return im.crop(bbox)

In [ ]:
im = get_pil_img(df['s3_link'][39])
get_rid_of_whitespace(im)

Check that white space was removed...

In [ ]:
get_pil_img(df['s3_link'][3]).size

In [ ]:
get_rid_of_whitespace(im).size

### **4. Convert de-whitespaced photo to JPEG to feed into color extractor**

In [6]:
def convert_to_jpeg_img_plug_in(removed_white_space_img):   

#     img = get_rid_of_whitespace(url)
    img2 = removed_white_space_img.crop((1,20,50,80))

    b = BytesIO()
    im.save(b, format="jpeg")
    img3 = Image.open(b)
    
    return img3

In [ ]:
convert_to_jpeg_img_plug_in(get_rid_of_whitespace(get_pil_img(df['s3_link'][39])))

### **5. Get colors from images**

In [7]:
from __future__ import print_function

def get_color_codes(im):
    
#     pil_img = get_pil_img(url)
#     white_space_removed_im = get_rid_of_whitespace(pil_img)
#     im = convert_to_jpeg_img_plug_in(white_space_removed_im)

    NUM_CLUSTERS = 5

#     print('reading image')
    im = im.resize((150, 150))      # optional, to reduce time
    ar = np.asarray(im)
    shape = ar.shape
    ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)

#     print('finding clusters')
    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
#     print('cluster centres:\n', codes)

    vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

    index_max = scipy.argmax(counts)                    # find most frequent
    peak = codes[index_max]
    colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
#     print('most frequent is %s (#%s)' % (peak, colour))
    
#     data = {}
#     data[url] = {'hex': colour, 'rgb': peak}
    
    for x in peak:
        if x > 225:
            return codes[index_max - 1]
        else:
            return codes[index_max]

### **6. Run**

In [ ]:
# D3: Filter to get the hex codes for the category, sort by them

In [8]:
def main(url):
    
    try:
        pil_image = get_pil_img(url)
#         im = get_rid_of_whitespace(pil_image)
#         converted_im = convert_to_jpeg_img_plug_in(im)

        color_code_dict = get_color_codes(pil_image)
    #     df = pd.DataFrame.from_dict(test_dict, orient='index')

        return color_code_dict
    
    except:
        pass

In [9]:
%time df['rgb'] = df['s3_link'].apply(main)

CPU times: user 13min 42s, sys: 14.6 s, total: 13min 57s
Wall time: 28min 44s


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4358 entries, 0 to 4357
Data columns (total 9 columns):
Unnamed: 0        4358 non-null int64
Unnamed: 0.1      4358 non-null int64
Unnamed: 0.1.1    4358 non-null int64
imageLink         4358 non-null object
category          4358 non-null object
cardName          4358 non-null object
s3_file_name      4358 non-null object
s3_link           4358 non-null object
rgb               4358 non-null object
dtypes: int64(3), object(6)
memory usage: 306.5+ KB


In [16]:
df.to_csv('hallmark_card_rgb_codes.csv')

In [20]:
df['rgb'][0]

array([234.01605314, 204.78272903,  73.98200941])

In [21]:
234+204+73

511

In [23]:
df['rgb_sum'] = df['rgb'].apply(lambda x: sum(x))

In [34]:
df_sorted = df.sort_values(by='rgb_sum', ascending=False).reset_index()

In [39]:
! pip install plotly --upgrade

    100% |████████████████████████████████| 41.5MB 92kB/s 
  Stored in directory: /Users/rosiebabbra/Library/Caches/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Error checking for conflicts.
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2897, in _dep_map
    return self.__dep_map
  File "/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2691, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2888, in _parsed_pkg_info
    return self._pkg_info
  File "/anaconda3/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2691, in __getattr__
    raise AttributeError(attr)


In [61]:
#final_df = df_sorted.loc[:, df_sorted.columns != 'rgb_sum']
final_df.to_csv('hallmark_card_rgb_codes_sorted.csv')

In [60]:
final_df

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,imageLink,category,cardName,s3_file_name,s3_link,rgb
0,3156,3156,3153,223,https://cdn-us-ec.yottaa.net/56213828312e58038...,Mother,celebrating-the-mom-you-are-mothers-card,celebrating-the-mom-you-are-mothers-card,https://greeting-card-images.s3.amazonaws.com/...,"[254.9530628480509, 254.93396976929196, 254.93..."
1,3434,3434,3436,506,https://cdn-us-ec.yottaa.net/56213828312e58038...,Mother,queen-for-a-day-mothers-day-card-root-599msi3836,queen-for-a-day-mothers-day-card-root-599msi3836,https://greeting-card-images.s3.amazonaws.com/...,"[254.9567766409932, 254.92148522818715, 254.83..."
2,3982,3982,4001,89,https://cdn-us-ec.yottaa.net/56213828312e58038...,Any+Woman,you-are-a-unicorn-queen-thinking-of-you-card,you-are-a-unicorn-queen-thinking-of-you-card,https://greeting-card-images.s3.amazonaws.com/...,"[254.84650184113625, 254.9431877958969, 254.89..."
3,3441,3441,3444,514,https://cdn-us-ec.yottaa.net/56213828312e58038...,Mother,embossed-heart-with-icons-and-mom-phrases-moth...,embossed-heart-with-icons-and-mom-phrases-moth...,https://greeting-card-images.s3.amazonaws.com/...,"[254.8933957952468, 254.8930530164534, 254.899..."
4,1870,1870,1848,152,https://cdn-us-ec.yottaa.net/56213828312e58038...,Husband,layered-heart-and-leaves-romantic-anniversary-...,layered-heart-and-leaves-romantic-anniversary-...,https://greeting-card-images.s3.amazonaws.com/...,"[254.90150730640892, 254.89943619836612, 254.8..."
5,1089,1089,1047,11,https://cdn-us-ec.yottaa.net/56213828312e58038...,Daughter,you-were-born-daughter-birthday-card,you-were-born-daughter-birthday-card,https://greeting-card-images.s3.amazonaws.com/...,"[254.95250987027637, 254.8994923857868, 254.78..."
6,3047,3047,3042,112,https://cdn-us-ec.yottaa.net/56213828312e58038...,Mother,flowers-wonderful-woman-birthday-card,flowers-wonderful-woman-birthday-card,https://greeting-card-images.s3.amazonaws.com/...,"[254.87655706430255, 254.8293120861856, 254.91..."
7,1721,1721,1715,19,https://cdn-us-ec.yottaa.net/56213828312e58038...,Husband,tree-with-a-couples-initials-fathers-day-card,tree-with-a-couples-initials-fathers-day-card,https://greeting-card-images.s3.amazonaws.com/...,"[254.89232444202503, 254.83287969515516, 254.8..."
8,2069,2069,2052,79,https://cdn-us-ec.yottaa.net/56213828312e58038...,Wife,ken-sheldon-every-little-moment-wife-card-root...,ken-sheldon-every-little-moment-wife-card-root...,https://greeting-card-images.s3.amazonaws.com/...,"[254.85921080849238, 254.85921080849238, 254.8..."
9,3129,3129,3125,195,https://cdn-us-ec.yottaa.net/56213828312e58038...,Mother,life-gave-me-you-mothers-day-card,life-gave-me-you-mothers-day-card,https://greeting-card-images.s3.amazonaws.com/...,"[254.89680908599243, 254.81308815575986, 254.7..."
